## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

In [6]:
import mne
test_raw = mne.io.read_raw_fif("datasets/avi/single/sub1_run-epo1.fif")
test_raw.shape

Opening raw data file datasets/avi/single/sub1_run-epo1.fif...
Isotrak not found


C:\Users\caio-\AppData\Local\Temp\ipykernel_6192\1489785573.py:2: RuntimeWarning: This filename (datasets/avi/single/sub1_run-epo1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  test_raw = mne.io.read_raw_fif("datasets/avi/single/sub1_run-epo1.fif")


ValueError: No raw data in c:\Users\caio-\Documents\patter-recog\SSVEP\datasets\avi\single\sub1_run-epo1.fif